## 데이터 하나 : 300차원의 벡터 5개 => 300차원의 벡터 1개
5개의 벡터에 대한 중심 벡터를 찾아 저장

In [4]:
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import warnings
warnings.filterwarnings('ignore')

C:\Users\user\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\user\anaconda3\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
C:\Users\user\anaconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


## 데이터 정제
진행 과정에서 요약문이 5단어 이하인 경우 키워드 추출에 실패 -> 정제 후 다시 저장 필요

In [93]:
with open('D:/playdata_mini_proj/data/vectors.pickle', 'rb') as file:
    vectors = pickle.load(file)
    
vectors.head()

,result
0,"[[0.022363545, -0.028911056, -0.040336788, 0.0..."
1,"[[-0.116084486, -0.016968077, 0.03961698, 0.08..."
2,"[[0.013123234, 0.0094847735, 0.019651216, 0.10..."
3,"[[-0.036596924, -0.054901063, -0.06675389, 0.0..."
4,"[[0.013123234, 0.0094847735, 0.019651216, 0.10..."


In [82]:
# 키워드 추출에 실패한 예
vectors['result'][693]

[]

In [94]:
# 추출실패한 경우의 원본 데이터 확인
rss=pd.read_csv('D:/playdata_mini_proj/data/RSS_for_keybert.csv', encoding='ANSI')
rss.iloc[693]

Unnamed: 0                                                          693
rgs_dt                                                         20230116
tck_iem_cd                                                         LXFR
til_ifo               The Zacks Analyst Blog Highlights Luxfer Holdi...
ctgy_cfc_ifo                                                 Technology
mdi_ifo                                                           Zacks
news_smy_ifo                                      For Immediate Release
rld_ose_iem_tck_cd                                OZK,LXFR,SNX,ALG,LKFN
url_ifo               https://www.nasdaq.com/articles/the-zacks-anal...
Name: 693, dtype: object

In [95]:
# 추출실패한 경우의 키워드 추출 데이터 확인
with open('D:/playdata_mini_proj/data/keybert_result.pickle', 'rb') as file:
    keybert_result = pickle.load(file)

In [85]:
# 추출 실패한 데이터의 인덱스 추출
count = 0
trash = []
for i in range(len(vectors)):
    if len(vectors['result'][i]) == 0:
        trash.append(i)
        count += 1
        
print('trash count', count)

trash count 1403


In [96]:
# 추출한 인덱스의 데이터 제거
for i in trash:
    vectors.drop(i,axis=0,inplace=True)
    rss.drop(i,axis=0,inplace=True)
    keybert_result.drop(i,axis=0,inplace=True)

In [97]:
# 인덱스 수정
vectors.reset_index(inplace=True)
rss.reset_index(inplace=True)
keybert_result.reset_index(inplace=True)

In [98]:
# 자동 추가된 이전 인덱스 정보를 담은 열 제거
vectors.drop('index',axis=1,inplace=True)
rss.drop('index',axis=1,inplace=True)
keybert_result.drop('index',axis=1,inplace=True)

In [93]:
# 결과 확인
vectors.iloc[693]

result    [[-0.027555251, 0.026801134, -0.050310906, 0.0...
Name: 702, dtype: object

In [94]:
vectors.head()

,result
0,"[[0.022363545, -0.028911056, -0.040336788, 0.0..."
1,"[[-0.116084486, -0.016968077, 0.03961698, 0.08..."
2,"[[0.013123234, 0.0094847735, 0.019651216, 0.10..."
3,"[[-0.036596924, -0.054901063, -0.06675389, 0.0..."
4,"[[0.013123234, 0.0094847735, 0.019651216, 0.10..."


In [95]:
rss.head()

,Unnamed: 0,rgs_dt,tck_iem_cd,til_ifo,ctgy_cfc_ifo,mdi_ifo,news_smy_ifo,rld_ose_iem_tck_cd,url_ifo
0,0,20230102,NGS,Hoak & Co. Urges NGS Board to Halt CEO Search ...,Investing,Fintel,"Fintel reports that Hoak Public Equities, LP, ...",NGS,https://www.nasdaq.com/articles/hoak-co.-urges...
1,1,20230102,PFX,Here's Why PhenixFIN (PFX) is Poised for a Tur...,"Stocks,Investing",Zacks,PhenixFIN has been beaten down lately with to...,PFX,https://www.nasdaq.com/articles/heres-why-phen...
2,2,20230102,TYG,My Advice? Sell These 2 Dividend Funds in 2023,"Markets,Stocks",BNK Invest,There's a disconnect setting up in the energy ...,"FEN,TYG",https://www.nasdaq.com/articles/my-advice-sell...
3,3,20230102,NVIV,France's InVivo to buy one of the world's olde...,"Commodities,BioTech,US Markets",Reuters,"Malteries Soufflet, a branch of French agribus...",NVIV,https://www.nasdaq.com/articles/frances-invivo...
4,4,20230102,FEN,My Advice? Sell These 2 Dividend Funds in 2023,"Markets,Stocks",BNK Invest,There's a disconnect setting up in the energy ...,"FEN,TYG",https://www.nasdaq.com/articles/my-advice-sell...


In [99]:
# 새로 저장
with open('D:/playdata_mini_proj/data/vectors_remove_trash.pickle', 'wb') as file:
        pickle.dump(vectors, file)
        
with open('D:/playdata_mini_proj/data/RSS_for_keybert_remove_trash.pickle', 'wb') as file:
        pickle.dump(rss, file)
        
with open('D:/playdata_mini_proj/data/keybert_result_remove_trash.pickle', 'wb') as file:
        pickle.dump(keybert_result, file)

## 차원 축소

In [100]:
# 데이터 불러오기
with open('D:/playdata_mini_proj/data/vectors_remove_trash.pickle', 'rb') as file:
    vectors = pickle.load(file)

In [104]:
# 벡터를 일정 각도만큼 회전시키는 함수 정의
from scipy.linalg import expm

def rotate_vector(vector, angle_degrees):
    # 각도를 라디안으로 변환
    angle_radians = np.radians(angle_degrees)

    # 회전 행렬 생성 (Identity 행렬)
    identity_matrix = np.eye(vector.shape[0])  # 행의 크기를 인자로 사용

    # 지수 행렬을 사용하여 회전 행렬 계산
    rotation_matrix = expm(angle_radians * identity_matrix)

    # 벡터를 열 벡터로 변환 (300x1 행렬)
    vector = np.array(vector).reshape(-1, 1)

    # 회전된 벡터 계산
    rotated_vector = np.dot(rotation_matrix, vector)

    # 회전된 벡터를 다시 1차원 배열로 변환
    rotated_vector = rotated_vector.flatten()

    return rotated_vector


In [107]:
# 벡터 간 각도 계산 & 회전시키는 함수 호출하는 재귀함수 생겅
def to_one_v(new, Vs, i):
    
    if i == 5:
        return new
    # 두 벡터의 내적을 계산
    dot_product = np.dot(new, Vs[i])

    # 각 벡터의 크기 계산
    magnitude1 = np.linalg.norm(new)
    magnitude2 = np.linalg.norm(Vs[i])

    # 코사인 유사도 계산
    similarity = dot_product / (magnitude1 * magnitude2)

    # 각도 계산 (라디안에서 도로 변환)
    angle_degrees = np.degrees(np.arccos(similarity))
    new = rotate_vector(new, angle_degrees/2)
    
    return to_one_v(new, Vs,i+1)

In [108]:
# 데이터 전체에 대하여 실행
new_v = pd.DataFrame()
new_v['result'] = ['nan'] * len(vectors)

for i in range(len(vectors)):
    new_v['result'][i] = to_one_v(vectors['result'][i][0], vectors['result'][i], 1)

In [110]:
# PICKLE 파일로 저장
with open('D:/playdata_mini_proj/data/vectors_dim_rdct.pickle', 'wb') as file: pickle.dump(new_v, file)